## Fundus Processing

### Source: MedIMeta

### [Link](https://www.woerner.eu/projects/medimeta/)

### [Download Link](https://zenodo.org/records/7884735)

### Type: Double Classification

In [ ]:
# NO LONGER NEEDED
# !pip install medimeta

In [ ]:
# dependencies setup

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# unecessary now

# !mkdir -p /content/data/MedIMeta
# !wget -O /content/data/MedIMeta/fundus.zip \
#   "https://zenodo.org/record/7884735/files/fundus.zip?download=1"

# !unzip /content/data/MedIMeta/fundus.zip -d /content/data/MedIMeta/

In [ ]:
# unecessary now

# !mkdir -p /content/drive/MyDrive/Research/data/MedIMeta
# !cp -r /content/data/MedIMeta/fundus \
#        /content/drive/MyDrive/Research/data/MedIMeta/

In [ ]:
# data loading

df = pd.read_csv("/content/drive/My Drive/Prof-Tang-Research/Data/MedIMeta/fundus/annotations.csv")
df.head()

,filepath,split,tasks/disease presence,tasks/disease labels,original_filepath,original_split,disease labels,disease presence,original_image_size
0,images/000000.tiff,train,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,DR,abnormal,"(2144, 1424)"
1,images/000001.tiff,train,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,DR,abnormal,"(2144, 1424)"
2,images/000002.tiff,train,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,DR,abnormal,"(2144, 1424)"
3,images/000003.tiff,train,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,MH|ODC,abnormal,"(2144, 1424)"
4,images/000004.tiff,train,1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,DR|LS,abnormal,"(2144, 1424)"


In [ ]:
df.columns

Index(['filepath', 'split', 'tasks/disease presence', 'tasks/disease labels',
       'original_filepath', 'original_split', 'disease labels',
       'disease presence', 'original_image_size'],
      dtype='object')

### Label(s) of interest: disease presence

In [ ]:
# generate subset of 3000 images
# find satisfactory floors (atleast 30%)

# split by abnormal and normal

df_abnormal = df[df['disease presence'] == 'abnormal']
df_normal = df[df['disease presence'] == 'normal']

abnormal_count = len(df_abnormal)
normal_count = len(df_normal)

print('abnormal proportion: ' + str(abnormal_count))
print('normal proportion: ' + str(normal_count))

# generate subset df w/ maximal floors

abnormal_needed = 3000 - normal_count

# get required number of abnormal entries
abnormal_subset = df_abnormal.iloc[:abnormal_needed]

# combine both
subset_df = pd.concat([df_normal, abnormal_subset], ignore_index=True)

print('Subset size:', len(subset_df))
print('Normal proportion:', len(df_normal)/3000)
print('Abnormal proportion:', len(abnormal_subset)/3000)

abnormal proportion: 2531
normal proportion: 669
Subset size: 3000
Normal proportion: 0.223
Abnormal proportion: 0.777


### NOTE: The floor here is 22.3% for normal images. 77.7% of the remaining images are abnormal

In [ ]:
# condition matching

conditions_full = [
    "Diabetic Retinopathy",                       # DR
    "Age-Related Macular Degeneration",          # ARMD
    "Macular Hole",                               # MH
    "Diabetic Neuropathy",                        # DN
    "Myopia",                                     # MYA
    "Branch Retinal Vein Occlusion",             # BRVO
    "Tessellated Fundus",                         # TSLN
    "Epiretinal Membrane",                        # ERM
    "Lattice Degeneration",                       # LS
    "Multiple Sclerosis",                         # MS
    "Central Serous Retinopathy",                 # CSR
    "Optic Disc Cupping",                         # ODC
    "Central Retinal Vein Occlusion",             # CRVO
    "Temporal Vein",                               # TV
    "Asteroid Hyalosis",                           # AH
    "Optic Disc Pit",                               # ODP
    "Optic Disc Edema",                              # ODE
    "Stargardt Disease",                                # ST
    "Anterior Ischemic Optic Neuropathy",                  # AION
    "Paracentral Tubular",                                  # PT
    "Retinal Tear",                                          # RT
    "Retinoschisis",                                         # RS
    "Central Retinal Sheath",                                # CRS
    "Exudative Detachment",                                  # EDN
    "Retinal Pigment Epithelial Changes",                    # RPEC
    "Lamellar Macular Hole",                                  # MHL
    "Retinitis Pigmentosa",                                    # RP
    "Cotton Wool Spot",                                        # CWS
    "Choroidal Blood / Bruising",                               # CB
    "Optic Disc Pit Maculopathy",                                # ODPM
    "Pre-retinal Hemorrhage",                                      # PRH
    "Myelinated Nerve Fibers",                                      # MNF
    "Hard Exudates",                                                 # HR
    "Central Retinal Artery Occlusion",                               # CRAO
    "Temporal Disc",                                                  # TD
    "Cystoid Macular Edema",                                           # CME
    "Posterior Cortical Cataract",                                      # PTCR
    "Chorioretinal Fold / Choroidal Fissure",                            # CF
    "Vitreous Hemorrhage",                                               # VH
    "Middle Cerebral Artery",                                             # MCA
    "Visual Sensation",                                                   # VS
    "Branch Retinal Artery Occlusion",                                     # BRAO
    "Plaque",                                                                # PLQ
    "Hyperpigmented Epithelial Defect / Deposit",                             # HPED
    "Collateral Vessel / Chorioretinal Lesion"                                  # CL
]

def map_values_to_conditions(lst):
  exhibited_conditions = []
  for i in range(len(lst)):
    if lst[i] == 1:
      exhibited_conditions.append(conditions_full[i])
  return exhibited_conditions

# # Example input
# lst = [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]

# # Example usage:
# mapping = map_values_to_conditions(lst)
# print(mapping)


In [ ]:
# control step 1 -> retrieval captions

import random
import ast

'''
two captions:
  1. this image depicts an (normal/abnormal) fundus
  2. this image depicts a (normal/abnormal) fundus with (disease presence)
'''

fundus_control_df = subset_df.copy()

def generate_control_retrieval_captions(disease_labels, disease_presence):
  # generate caption 1
  caption_1 = "This image depicts a " + disease_presence + " fundus"

  if isinstance(disease_labels, str):
        disease_labels = ast.literal_eval(disease_labels)

  # get diseases
  diseases = map_values_to_conditions(disease_labels)

  # if multiple diseases are present, pick one at random
  if len(diseases) > 1:
    disease_labels = random.choice(diseases)
  elif len(diseases) == 1:
    disease_labels = diseases[0]
  else:
    disease_labels = "no disease"

  caption_2 = "This image depicts a " + disease_presence + " fundus with " + disease_labels

  return [caption_1, caption_2]

# add as a new col in fundus_control_df

fundus_control_df['retrieval_captions'] = fundus_control_df.apply(lambda row: generate_control_retrieval_captions(row['tasks/disease labels'], row['disease presence']), axis=1)
fundus_control_df.head()


,filepath,split,tasks/disease presence,tasks/disease labels,original_filepath,original_split,disease labels,disease presence,original_image_size,retrieval_captions
0,images/000009.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag..."
1,images/000015.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag..."
2,images/000016.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag..."
3,images/000020.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag..."
4,images/000027.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag..."


In [ ]:
# control step 2

In [ ]:
# control step 3 -> MCQ

'''
two options:
  1. this image depicts a (normal/abnormal) fundus
  2. this image depicts a (normal/abnormal) fundus
'''

def generate_control_mcq_options(disease_presence):
  true = disease_presence
  false = "abnormal" if disease_presence == "normal" else "normal"

  # generate true question first
  true_question = "This image depicts a " + disease_presence + " fundus"

  # generate incorrect question next
  false_question = "This image depicts a " + false + " fundus"

  return [true_question, false_question]

# add a new col in fundus_control_df
# note that the first mcq option is always correct

fundus_control_df['mcq_options'] = fundus_control_df['disease presence'].apply(generate_control_mcq_options)
fundus_control_df.head()


,filepath,split,tasks/disease presence,tasks/disease labels,original_filepath,original_split,disease labels,disease presence,original_image_size,retrieval_captions,mcq_options
0,images/000009.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag...","[This image depicts a normal fundus, This imag..."
1,images/000015.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag...","[This image depicts a normal fundus, This imag..."
2,images/000016.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag...","[This image depicts a normal fundus, This imag..."
3,images/000020.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag...","[This image depicts a normal fundus, This imag..."
4,images/000027.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)","[This image depicts a normal fundus, This imag...","[This image depicts a normal fundus, This imag..."


In [ ]:
# control step 4

In [ ]:
# test step 1 -> retrieval captions

'''
two captions:
  1. this image depicts a fundus that is NOT (normal/abnormal)
  2. this image depicts a fundus that is NOT (normal/abnormal) with (disease presence)
'''

fundus_negated_df = subset_df.copy()

def generate_negated_retrieval_captions(disease_labels, disease_presence):
  # for each row generate two captions, using complement disease_presence
  if disease_presence == "normal":
    disease_presence = "abnormal"
  else:
    disease_presence = "normal"

  caption_1 = "This image depicts a fundus that is not " + disease_presence

  if isinstance(disease_labels, str):
        disease_labels = ast.literal_eval(disease_labels)

  # get diseases
  diseases = map_values_to_conditions(disease_labels)

  # if multiple diseases are present, pick one at random
  if len(diseases) > 1:
    disease_labels = random.choice(diseases)
  elif len(diseases) == 1:
    disease_labels = diseases[0]
  else:
    disease_labels = "no disease"

  caption_2 = "This image depicts a fundus with " + disease_labels + " that is not " + disease_presence

  return [caption_1, caption_2]

# add as a new col in fundus_negated_df

fundus_negated_df['retrieval_captions'] = fundus_negated_df.apply(lambda row: generate_negated_retrieval_captions(row['tasks/disease labels'], row['disease presence']), axis=1)
fundus_negated_df.head()


,filepath,split,tasks/disease presence,tasks/disease labels,original_filepath,original_split,disease labels,disease presence,original_image_size,retrieval_captions
0,images/000009.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...
1,images/000015.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...
2,images/000016.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...
3,images/000020.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...
4,images/000027.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...


In [ ]:
# test step 2

In [ ]:
# test step 3

'''
two options:
  1. this image does not depict a (normal/abnormal) fundus
  2. this image does not depict a (normal/abnormal) fundus
'''

def generate_negated_mcq_options(disease_presence):
  true = disease_presence
  false = "abnormal" if disease_presence == "normal" else "normal"

  # generate true question first
  true_question = "This image does not depict a " + false + " fundus"

  # generate incorrect question next
  false_question = "This image does not depict a " + true + " fundus"

  return [true_question, false_question]

# add a new col in fundus_control_df
# note that the first mcq option is always correct

fundus_negated_df['mcq_options'] = fundus_negated_df['disease presence'].apply(generate_negated_mcq_options)
fundus_negated_df.head()

,filepath,split,tasks/disease presence,tasks/disease labels,original_filepath,original_split,disease labels,disease presence,original_image_size,retrieval_captions,mcq_options
0,images/000009.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...,"[This image does not depict a abnormal fundus,..."
1,images/000015.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...,"[This image does not depict a abnormal fundus,..."
2,images/000016.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...,"[This image does not depict a abnormal fundus,..."
3,images/000020.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...,"[This image does not depict a abnormal fundus,..."
4,images/000027.tiff,train,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A. RFMiD_All_Classes_Dataset/1. Original Image...,train,NaN,normal,"(2144, 1424)",[This image depicts a fundus that is not abnor...,"[This image does not depict a abnormal fundus,..."


In [ ]:
# test step 4

In [ ]:
# temp download the csv

fundus_control_df.to_csv('/content/drive/My Drive/Prof-Tang-Research/Data/MedIMeta/fundus_control.csv', index=False)
fundus_negated_df.to_csv('/content/drive/My Drive/Prof-Tang-Research/Data/MedIMeta/fundus_negated.csv', index=False)

In [ ]:
# get freq of disease prescence across first 300 entries
df.head(3000)['disease presence'].value_counts()

,count
disease presence,
abnormal,2433
normal,567


In [ ]:
from medimeta import MedIMeta

drive_root = "/content/drive/MyDrive/Research/data/MedIMeta"
ds = MedIMeta(drive_root, "fundus", "disease presence")

img, lbl = ds[0]
print("Shape:", img.shape)      # → torch.Size([1, 224, 224])
print("Label:", lbl)            # → 0 (normal) or 1 (abnormal)


Shape: torch.Size([3, 224, 224])
Label: tensor([1])


In [ ]:
from medimeta import MedIMeta

drive_root = "/content/drive/MyDrive/Research/data/MedIMeta"

# Confirm which tasks are accepted
for task in ["disease presence", "disease type"]:
    try:
        _ = MedIMeta(drive_root, "fundus", task)
        print(f"✔️ Task available: '{task}'")
    except ValueError as e:
        print(f"❌ Not available: '{task}' –", e)

# Load the dataset properly (choose one task)
ds = MedIMeta(drive_root, "fundus", "disease type")

# Try pulling a sample
img, lbl = ds[0]
print("Image shape:", img.shape)         # Should be torch.Size([1, 224, 224])
print("Label shape:", lbl.shape, "→", lbl)  # For multi-label: ~45-length vector


✔️ Task available: 'disease presence'
❌ Not available: 'disease type' – No task with name 'disease type' found for dataset 'Fundus Multi-disease'


ValueError: No task with name 'disease type' found for dataset 'Fundus Multi-disease'

In [ ]:
from medimeta import MedIMeta
drive_root = "/content/drive/MyDrive/Research/data/MedIMeta"

for task in ["Disease presence", "Disease labels"]:
    try:
        MedIMeta(drive_root, "fundus", task)
        print(f"✔️ Task available: '{task}'")
    except ValueError as e:
        print(f"❌ Not available: '{task}' ->", e)

❌ Not available: 'Disease presence' -> No task with name 'Disease presence' found for dataset 'Fundus Multi-disease'
❌ Not available: 'Disease labels' -> No task with name 'Disease labels' found for dataset 'Fundus Multi-disease'


In [ ]:
# !pip install medimeta
from medimeta import MedIMeta

drive_root = "/content/drive/MyDrive/Research/data/MedIMeta"

# Check available tasks
# (this raises a ValueError listing valid names)
try:
    MedIMeta(drive_root, "fundus", "dummy")
except ValueError as e:
    print("Valid tasks >", e)

# Example: load multi-label disease classification
ds = MedIMeta(drive_root, "fundus", "Disease")
img, lbl = ds[0]
print(img.shape, lbl)

Valid tasks > No task with name 'dummy' found for dataset 'Fundus Multi-disease'


ValueError: No task with name 'Disease' found for dataset 'Fundus Multi-disease'

In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/Research/data/fundus.zip'
extract_to = '/content/drive/MyDrive/Research/data/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
from medimeta import MedIMeta

data_path = "/content/drive/MyDrive/Research/data"

# List all supported datasets and their tasks
from medimeta.medimeta import DATASETS
print("Available datasets and tasks:")
for ds in DATASETS:
    print(ds['dataset_id'])
    for task in ds['tasks']:
        print(" ", task['task_name'])

ImportError: cannot import name 'DATASETS' from 'medimeta.medimeta' (/usr/local/lib/python3.11/dist-packages/medimeta/medimeta.py)

In [ ]:
!unzip /content/drive/MyDrive/Research/data/fundus.zip -d /content/drive/MyDrive/Research/data/


Archive:  /content/drive/MyDrive/Research/data/fundus.zip
replace /content/drive/MyDrive/Research/data/fundus/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!find /content/drive/MyDrive/Research/data/ -maxdepth 2 | sed -e 's/^/📁 /'

📁 /content/drive/MyDrive/Research/data/
📁 /content/drive/MyDrive/Research/data/fundus.zip
📁 /content/drive/MyDrive/Research/data/fundus_raw
📁 /content/drive/MyDrive/Research/data/fundus_raw/LICENSE
📁 /content/drive/MyDrive/Research/data/fundus_raw/images.hdf5
📁 /content/drive/MyDrive/Research/data/fundus_raw/annotations.csv
📁 /content/drive/MyDrive/Research/data/fundus_raw/teaser.png
📁 /content/drive/MyDrive/Research/data/fundus_raw/info.yaml
📁 /content/drive/MyDrive/Research/data/fundus
📁 /content/drive/MyDrive/Research/data/fundus/LICENSE
📁 /content/drive/MyDrive/Research/data/fundus/images
📁 /content/drive/MyDrive/Research/data/fundus/images.hdf5
📁 /content/drive/MyDrive/Research/data/fundus/splits
📁 /content/drive/MyDrive/Research/data/fundus/original_splits
📁 /content/drive/MyDrive/Research/data/fundus/task_labels
📁 /content/drive/MyDrive/Research/data/fundus/annotations.csv
📁 /content/drive/MyDrive/Research/data/fundus/teaser.png
📁 /content/drive/MyDrive/Research/data/fundus/info

In [ ]:
from medimeta import MedIMeta
import os

data_root = "/content/drive/MyDrive/Research/data/MedIMeta"
fundus_folder = os.path.join(data_root, "fundus")

# Load the dataset's metadata JSON
dataset_meta = MedIMeta._load_metadata(fundus_folder)
print("Available tasks:", list(dataset_meta["tasks"].keys()))

AttributeError: type object 'MedIMeta' has no attribute '_load_metadata'

In [ ]:
from medimeta import MedIMeta

data_root = "/content/drive/MyDrive/Research/data/MedIMeta"
all_tasks = MedIMeta(data_root, "fundus").tasks
print("Available tasks:", all_tasks)

TypeError: MedIMeta.__init__() missing 1 required positional argument: 'task_name'

In [ ]:
from medimeta import MedIMeta

data_root = "/content/drive/MyDrive/Research/data/MedIMeta"
dataset = MedIMeta(data_root, "fundus", "Disease")  # Capital "D" for task name

img, lbl = dataset[0]
print(img.shape)  # Expect torch.Size([1, 224, 224])
print(lbl)

ValueError: No task with name 'Disease' found for dataset 'Fundus Multi-disease'

In [ ]:
from medimeta import MedIMeta

data_path = "/content/drive/MyDrive/Research/data"  # Should contain 'fundus/' after extraction

# Load the fundus dataset's "Disease" task
dataset = MedIMeta(data_path, "fundus", "Disease")

sample_img, sample_label = dataset[0]
print(sample_img.shape)      # Expected: torch.Size([1, 224, 224])
print(sample_label)

ValueError: No task with name 'Disease' found for dataset 'Fundus Multi-disease'

In [ ]:
from medimeta import MedIMeta

# Set the path to where you uploaded the MedIMeta data in your Google Drive
data_path = "/content/drive/MyDrive/Research/data/fundus_raw"  # adjust this path as needed

# Load the OCT dataset for the 'Disease' task
dataset = MedIMeta(data_path, "fundus", "Disease")

# Access the first sample
sample_img, sample_label = dataset[0]
print(sample_img.shape)  # Expected: torch.Size([1, 224, 224])
print(sample_label)

ValueError: Dataset with ID 'fundus' not found. Available datasets (name and ID): []